## Imports des librairies

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score


## Génération et préparation des données

In [ ]:
# Génération des données
X, y = make_moons(n_samples=1000, noise=0.2, random_state=42)
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Conversion en tenseurs PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)


## Conversion des données en tenseurs (`torch.tensor`)

PyTorch attend des `torch.Tensor`, pas des tableaux NumPy. On convertit donc :
- les données d'entrée en `float32`
- les labels en `long` (entiers), requis pour `CrossEntropyLoss`

```python
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)



---
## Exercice 1 — Définir un MLP

Complétez la classe ci-dessous pour créer un MLP avec :
- une couche cachée de **32 neurones**
- une activation **ReLU**
- une sortie à **2 neurones** (car 2 classes)


In [ ]:
class SimpleMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, ___)        # À compléter
        self.fc2 = nn.Linear(___, ___)      # À compléter

    def forward(self, x):
        x = F.___(self.fc1(x))              # À compléter
        return self.fc2(x)


## Exercice 2 — Entraînement du modèle

Complétez la boucle pour afficher la **loss** et l'**accuracy** tous les 10 epochs.


In [ ]:
# Initialisation du modèle
model = ___            # Créer une instance du MLP défini précédemment
criterion = ___        # Choisir une fonction de perte adaptée à un problème de classification
optimizer = ___        # Choisir un optimiseur avec un taux d'apprentissage de 0.01

# Paramètres de la boucle d'entraînement
n_epochs = ___         # Nombre d’époques (ex : 100)
batch_size = ___       # Taille des batchs (ex : 64)
train_losses = []

# Boucle d'entraînement
for epoch in range(n_epochs):

    # tirage d'un ordre aléatoire de présentation des exemple dans chaque epoch
    permutation = torch.randperm(X_train_tensor.size()[0])
    epoch_loss = 0
    correct = 0

    for i in range(0, X_train_tensor.size()[0], batch_size):
        indices = permutation[i:i + batch_size]

        # Extraire les batchs de données et de labels
        batch_x = ___
        batch_y = ___

        # Calcul de la sortie du réseau
        outputs = ___

        # Calcul de la perte
        loss = ___

        # Mise à jour des poids
        optimizer.___()       # Reset du gradient
        ___.backward()        # Calcul du gradient
        ___.step()            # Mise à jour des poids

        # Suivi de la perte
        epoch_loss += ___

        # Prédictions et comptage des bonnes réponses
        _, predicted = torch.max(outputs, 1)
        correct += ___

    # Calcul de l'accuracy globale pour l'époque
    accuracy = ___

    train_losses.append(epoch_loss)

    # Affichage régulier des résultats
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}, Loss = {___}, Accuracy = {___}")


## Pourquoi utiliser `torch.no_grad()` ?

Lors de l'évaluation (test), on n’a pas besoin de gradients. `torch.no_grad()` permet de :
- désactiver le calcul automatique des gradients,
- accélérer le calcul,
- économiser de la mémoire.

```python
with torch.no_grad():
    outputs = model(X_test_tensor)



---

### **Évaluation du modèle**


In [ ]:
# Évaluation du modèle (à compléter si besoin)
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)
    acc = accuracy_score(y_test, predicted.numpy())
    print(f"Test accuracy : {acc:.2f}")

---
### Affichage de la frontière de décisions

In [ ]:
def plot_decision_boundary(model, X, y):
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 300), np.linspace(y_min, y_max, 300))
    grid = np.c_[xx.ravel(), yy.ravel()]
    grid_t = torch.tensor(grid, dtype=torch.float32)

    with torch.no_grad():
        preds = model(grid_t)
        _, pred_labels = torch.max(preds, 1)

    plt.figure(figsize=(6, 5))
    plt.contourf(xx, yy, pred_labels.numpy().reshape(xx.shape), cmap='coolwarm', alpha=0.6)
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap='coolwarm', edgecolors='k')
    plt.title("Frontière de décision du modèle")
    plt.show()

# Appel de la fonction avec les données de test
plot_decision_boundary(model, X_test, y_test)


## Affichage des points mal classés

In [ ]:
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)
    incorrect = predicted != y_test_tensor
    incorrect_indices = np.where(incorrect.numpy())[0]

print(f"Nombre de points mal classés : {len(incorrect_indices)}")

plt.figure(figsize=(6, 5))
plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap='coolwarm', edgecolors='k', alpha=0.3, label='Vrai label')
plt.scatter(X_test[incorrect_indices, 0], X_test[incorrect_indices, 1], c='black', label='Erreur')
plt.legend()
plt.title("Points mal classés par le modèle")
plt.show()


## Exercice 3 — Comparaison de différentes architectures

On souhaite comparer :
- un modèle **sous-ajusté** (trop simple)
- un modèle **sur-ajusté** (trop complexe)
- un modèle **régularisé** (avec Dropout)

Complétez les architectures suivantes puis entraînez-les comme dans l'Exercice 2.
Comparez leur précision, courbe de perte et frontière de décision.

Vous pourrez ensuite tester d'autres architectures encore.


In [ ]:
# Modèle sous-ajusté : une seule couche linéaire
class UnderfitMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(2, ___)  # À compléter

    def forward(self, x):
        return ___                   # À compléter


In [ ]:
# Modèle sur-ajusté : deux couches larges
class OverfitMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, ___)    # À compléter
        self.fc2 = nn.Linear(___, ___) # À compléter
        self.fc3 = nn.Linear(___, 2)

    def forward(self, x):
        x = F.___(self.fc1(x))         # À compléter
        x = F.___(self.fc2(x))         # À compléter
        return self.fc3(x)


In [ ]:
# Modèle régularisé : avec Dropout
class DropoutMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, ___)        # À compléter
        self.dropout = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(___, 2)

    def forward(self, x):
        x = F.___(self.fc1(x))              # À compléter
        x = self.dropout(x)
        return self.fc2(x)


## Exercice 4 — Calcul de la loss de validation

Complétez le code ci-dessous pour :
- ajouter une séparation du jeu d’entraînement en **train** et **validation**
- calculer la **loss de validation** à chaque époque
- afficher la loss de validation tous les 10 epochs

Cela permet de suivre le comportement du modèle sur des données non vues pendant l'entraînement.


In [ ]:
# Nouvelle séparation : X_train → X_subtrain + X_val
X_subtrain, X_val, y_subtrain, y_val = train_test_split(
    X_train_tensor, y_train_tensor, test_size=0.2, random_state=42
)

# Entraînement sur X_subtrain
# Ajouter calcul de la loss sur X_val à chaque epoch
val_losses = []

for epoch in range(n_epochs):
    # boucle d'entraînement classique (sur X_subtrain)
    # ...

    # Partie à compléter : calcul de la loss de validation
    with torch.no_grad():
        val_outputs = model(___)             # À compléter
        val_loss = criterion(val_outputs, ___)  # À compléter
        val_losses.append(val_loss.item())

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}, Validation loss = {___}")  # À compléter


Afficher la loss en apprentissage et en validation en fonction des epochs

## Pour aller plus loin

Dans un nouveau notebook, définir un nouveau MLP pour travailler sur les données ``digits`̀  de scikit-learn